In [1]:
import gradio as gr
from grammar_error_correction import GrammarErrorCorrector

from token_realignment import RobertaTokenWordAlignment
from token_realignment import BertTokenWordAlignment

/home/estudiante/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-26 12:18:26.850359: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 12:18:26.900892: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 12:18:27.772986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Ten

In [3]:
def load_model(model_name):
    MODEL_PATH = f"../Project_Testing/{model_name}"
    if model_name.startswith("roberta"):
        TOKEN_ALIGNMENT = RobertaTokenWordAlignment()
    elif model_name.startswith("bert"):
        TOKEN_ALIGNMENT = BertTokenWordAlignment()
    return GrammarErrorCorrector(MODEL_PATH, TOKEN_ALIGNMENT)

# Initialize with a default model
gec = load_model("bert_ner_model")

def predict(text):
    return gec.correct_sentence(text)

def verify_text(input_text, model_name):
    global gec
    try:
        # Load the selected model
        gec = load_model(model_name)
        return gec.correct_sentence(input_text)
    except Exception as e:
        return f"Error al procesar el texto: {str(e)}"

def clear_text():
    return "", ""

custom_css = """
#header {
    text-align: left;
    margin-left: 10px;
}

#university-text {
    font-size: 20px;
    font-weight: bold;
}

#ml-text {
    font-size: 14px;
    color: #555;
}
"""

with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        with gr.Column():
            gr.HTML("""
                <div id='header'>
                    <div id='university-text'>Universidad de Los Andes</div>
                    <div id='ml-text'>Machine Learning Techniques. MISIS-4219</div>
                </div>
            """)
        with gr.Column():
            gr.Markdown("<h1 style='text-align: center;'>Spanish Grammar Verification Application</h1>")
        with gr.Column():
            pass
    with gr.Row():
        model_selector = gr.Dropdown(choices=["bert_ner_model", "roberta_ner_model"], value="bert_ner_model", label="Select Model")
    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(lines=10, placeholder="Escribe o pega tu texto aquí", label="Introduzca texto")
        
        with gr.Column():
            result_output = gr.Textbox(lines=10, interactive=False, label="Resultado revisión")
    
    with gr.Row():
        check_btn = gr.Button("Verify")
        clean_btn = gr.Button("Clean")
    
    check_btn.click(fn=verify_text, inputs=[text_input, model_selector], outputs=result_output)
    clean_btn.click(fn=clear_text, inputs=None, outputs=[text_input, result_output])

demo.launch(server_name="0.0.0.0", server_port=7861)
# demo.launch(server_port=7681) # local check

Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.
